In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

Going to write a function that will take in the locations of five balls located on the corners of the strike zone (in pixel coordinates), as well as the locations of the top and bottom of the zone (again in pixel coordinates) to establish the bounds of the strike zone. We will then calculate whether the pitch crossed into the zone or not.

In [4]:
def pixel_to_world_coordinate(camera_matrix,point_list):
    # Takes in the camera matrix (numpy array)
    # point_list: list of tuples that contain the u, and v, dimensions of the bounding box and true diameter of the object
    #returns a list of tuples, where each element of the list is a tuple containing the x,y,z coordinates of the center of the object. From the POV of the camera, x refers to movement left and right
    # y refers to movement up and down, and z is depth (closer and farther from the camera)

    world_coordinate_list = []

    for ball in point_list:
        bounding_box = np.array([ball[0],ball[1]]) #create an array containing the lengths of the bounding box
        Z = ( camera_matrix[0,0]*ball[2] )/np.average(bounding_box) #Calculate Z coordinate. Since these are balls, the boxes should be square. If they're not, we take 
                                                                    # the average of the length of each side to be the size of the ball
        Y = ( ball[1]-camera_matrix[1,2] )/camera_matrix[1,1] #calculate Y coordinate
        X = ( ball[0] - camera_matrix[0,2] )/camera_matrix[0,0] #calculate x coordinate
        world_coordinate_list.append( (X,Y,Z) )
    return world_coordinate_list #list of tuples

Now we are going to make a function that will take in the lengths of the bounding boxes of the balls on the corners of the plate and convert those to world coordinates to get the bounds on the strike zone. 

In [7]:
def build_zone(world_coordinate_list):
    #The world coordinate list should be ordered as follows: The 1st entry should be the position of the ball that is at the back (the point) of the plate, from there, 
    #rotate counter clockwise until every ball on the plate is accounted for. The sixth entry will be the lower bound for the zone, the seventh entry will be the upper bound of the zone
    #We're also going to average over the coordinates of the corners of the zone to mitigate errors in strike zone dimensions

     #/\
    #|  |
    #____
    #Front of the zone
    #Front plane of the zone length in x coordinate
    front_plane_length_x = world_coordinate_list[2][0] - world_coordinate_list[3][0] 
    back_plane_length_x = world_coordinate_list[1][0] - world_coordinate_list[4][0] 

    x_length_front = ( front_plane_length_x + back_plane_length_x )/2 #average of the 2 lengths

    #y length of the front zone

    front_plane_length_y = world_coordinate_list[1][1] - world_coordinate_list[2][1] 
    back_plane_length_y = world_coordinate_list[4][1] - world_coordinate_list[3][1] 

    y_length_front = ( front_plane_length_y + back_plane_length_y )/2 #average of the 2 lengths

    z_length = world_coordinate_list[6][2] - world_coordinate_list[5][2] #Height of the front plane of the zone

    #Now for the back of the zone where we have curvature. We want to return the slope of the rear of the zone so we can use that for ball and strike checking later
    back_zone_left_edge_slope = (world_coordinate_list[0][1] - world_coordinate_list[4][1] )/( world_coordinate_list[0][0] - world_coordinate_list[4][0] ) #change in y over change in x babay

    back_zone_right_edge_slope = (world_coordinate_list[0][1] - world_coordinate_list[1][1] )/( world_coordinate_list[1][0] - world_coordinate_list[0][0] ) #change in y over change in x babay
    #Also want x intercepts of both of these lines

    intercept_left = world_coordinate_list[0][1] - back_zone_left_edge_slope*world_coordinate_list[0][0]
    intercept_right = world_coordinate_list[0][1] - back_zone_right_edge_slope*world_coordinate_list[0][0]
    #We also want the y dimension length of the back of the zone. So the tip of the plate to the back plane of the zone
    y_length_back = ( (world_coordinate_list[0][1] - world_coordinate_list[4][1] ) + (world_coordinate_list[0][1] - world_coordinate_list[1][1] ) )/2 #average over 2 possible lengths


    return (x_length_front, y_length_front, z_length, y_length_back, back_zone_left_edge_slope, back_zone_right_edge_slope, intercept_left, intercept_right) # return a tuple of the bounding edges of the zone


In [ ]:
def check_strike(ball_position, zone_bounds):
    #ball position: tuple of the world coordinates of the ball
    #Zone bounds: list of tuples that contains the center positions of each of the balls used to mark the edges of the strike zone.
    #The 1st entry should be the position of the ball that is at the back (the point) of the plate, from there, 
    #rotate counter clockwise until every ball on the plate is accounted for. The sixth entry will be the lower bound for the zone, the seventh entry will be the upper bound of the zone
    #We're also going to average over the coordinates of the corners of the zone to mitigate errors in strike zone dimensions
    
    zone_dims = build_zone(zone_bounds)
    y_check_front = ball_position[1] > zone_bounds[3][1] and ball_position[1] < zone_bounds[3][1] + zone_dims[1] #Check if ball is outside the front of zone to start 
    if y_check_front == True: #Ball is in the front of the zone
        x_check_front = ball_position[0] > zone_bounds[3][0] and ball_position[0] < zone_bounds[3][0] + zone_dims[0] #Check if ball is in x range of the front of the zone
        z_check_front = ball_position[2] > zone_bounds[5][2] and ball_position[2] < zone_bounds[6][2] #check if ball is in the z range of the zone
        if x_check_front == True and z_check_front == True:
            strike = 1
            return strike

    y_check_back = ball_position[1] > zone_bounds[4][1] and ball_position[1] < zone_bounds[4][1] + zone_dims[3] #check if ball y coordinate is in the back portion of the zone

    if y_check_back == True:
        #Using the x intercepts of the lines calculated in zone_dims 
        x_check_back = ball_position[0] > ( ball_position[1] - zone_dims[6] ) / (zone_dims[4]) and ball_position[0] < ( ball_position[1] - zone_dims[7] ) / (zone_dims[5])
        z_check_back = ball_position[2] > zone_bounds[5][2] and ball_position[2] < zone_bounds[6][2] #check if ball is in the z range of the zone
        if x_check_back == True and z_check_back == True:
            strike = 1
            return strike
    strike = 0
    return strike

We also need to establish which of the balls in the video are actually used to bound the zone, and which is the ball that is actually being pitched. 

Possible options:
1) Divide up screen into sections, pick the balls that are the farthest left in terms of their pixel/world positions, and the balls that are the lowest in the screen. Will need a flag to check if batter is right or left handed so not great
2) calculate the difference in sizes between the bounding boxes from frame to frame? If the bounding box changes significantly in size, then we know that the bounding box corresponds to a pitch. Relies on the assumption that umpire head movement is minimal. 


In [ ]:
def find_bounding_area(bounding_box_lengths):
    #takes in a list of bounding box dimensions, and determines which balls are moving. Would need to be called on each frame of the video
    for 